In [1]:
import requests
import spacy
import json

2023-11-13 23:25:57.446649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
nlp = spacy.load("en_core_web_sm")

API_KEY = open('apikey').read()

baltimore_harbor = {
    "circle": {
        "center": {
            "latitude": 39.2820552,
            "longitude": -76.6051076
        },
        "radius": 500
    }
}

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4.0 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [10]:
def nearby_places(locationRestriction, url="https://places.googleapis.com/v1/places:searchNearby"):
    body = {
        "maxResultCount": 10,
        "locationRestriction": locationRestriction
    }

    headers = {
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.editorialSummary"
    }
    
    r = requests.post(url, headers=headers, json=body)
    return r.json()

In [11]:
places = nearby_places(baltimore_harbor)
# places = json.load(open("places"))
places

{'places': [{'displayName': {'text': 'National Aquarium',
    'languageCode': 'en'},
   'editorialSummary': {'text': 'Iconic landmark offering marine-life exhibits, a dolphin show, shark tank & walk-through rainforest.',
    'languageCode': 'en'}},
  {'displayName': {'text': 'Baltimore Marriott Waterfront',
    'languageCode': 'en'},
   'editorialSummary': {'text': 'Modern high-end hotel with a laid-back American restaurant, a 24/7 fitness center & a sauna.',
    'languageCode': 'en'}},
  {'displayName': {'text': 'Four Seasons Hotel Baltimore',
    'languageCode': 'en'},
   'editorialSummary': {'text': 'A high-end property offering refined rooms & suites, plus a posh day spa, 2 pools & stylish dining.',
    'languageCode': 'en'}},
  {'displayName': {'text': 'Baltimore Aquarium', 'languageCode': 'en'}},
  {'displayName': {'text': 'Rusty Scupper Restaurant & Bar',
    'languageCode': 'en'},
   'editorialSummary': {'text': 'Local landmark serving upscale regional seafood dishes in a water

In [6]:
reviews = {p['displayName']['text']: [i['text']['text'] for i in p['reviews']] for p in places['places']}

In [9]:
from collections import Counter

reviews_counter = {}
for n, rs in reviews.items():
    adjs = [token.lemma_ for r in rs for token in nlp(r) if token.pos_ == "ADJ"]
    reviews_counter[n] = Counter(adjs)
reviews_counter

{'National Aquarium': Counter({'other': 3,
          'big': 3,
          'cool': 3,
          'great': 3,
          'hard': 2,
          'super': 2,
          'clean': 2,
          'worth': 2,
          'beautiful': 2,
          'busy': 2,
          'fantastic': 2,
          'more': 2,
          'happy': 2,
          'few': 2,
          'good': 2,
          'vertical': 2,
          'impressive': 1,
          'surprised': 1,
          'modern': 1,
          'easy': 1,
          'many': 1,
          'amazing': 1,
          'entire': 1,
          'little': 1,
          'confusing': 1,
          'wide': 1,
          'tropical': 1,
          'worthy': 1,
          'friendly': 1,
          'underwater': 1,
          'huge': 1,
          'fun': 1,
          'awesome': 1,
          'sure': 1,
          'jellyfish': 1,
          'hidden': 1,
          'favorite': 1,
          'unimpressed': 1,
          'most': 1,
          'small': 1,
          'various': 1,
          'geographical': 1,
      

In [ ]:
# place photos: https://developers.google.com/maps/documentation/places/web-service/place-photos

In [17]:
from openai import OpenAI

In [18]:
openapikey = open("openapikey").read()

In [22]:
client = OpenAI(api_key=openapikey)

BadRequestError: Error code: 400 - {'error': {'message': "You uploaded an unsupported image. Please make sure your image is below 20 MB in size and is of one the following formats: ['png', 'jpeg', 'gif', 'webp'].", 'type': 'invalid_request_error', 'param': None, 'code': 'sanitizer_server_error'}}

In [24]:
import base64
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


base64_image = encode_image("test.png")

In [30]:
response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Use exactly ten adjectives and no more words to describe the buildings in image."},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/9/9c/BaltimoreNationalAquarium.JPG",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason=None, index=0, message=ChatCompletionMessage(content='Modern, geometric, reflective, colorful, angular, waterfront, prominent, varied, textured, industrial.', role='assistant', function_call=None, tool_calls=None), finish_details={'type': 'stop', 'stop': '<|fim_suffix|>'})


In [27]:
# headers = {
#   "Content-Type": "application/json",
#   "Authorization": f"Bearer {openapikey}"
# }

# payload = {
#   "model": "gpt-4-vision-preview",
#   "messages": [
#     {
#       "role": "user",
#       "content": [
#         {
#           "type": "text",
#           "text": "What’s in this image?"
#         },
#         {
#           "type": "image_url",
#           "image_url": {
#             "url": f"data:image/jpeg;base64,{base64_image}"
#           }
#         }
#       ]
#     }
#   ],
#   "max_tokens": 300
# }

# response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

# print(response.json())

{'error': {'message': 'Something went wrong processing one of your images.', 'type': 'server_error', 'param': None, 'code': None}}
